# Components of Sea Level

In [1]:
# Eventually stuff will go here

Hi, I am stuff.

## Following Merrifield, Firing & Marra:
This technique uses the Peaks Over Treshold (POT) technique to obtain extrema. This treshold is based on annual climatologies, obtained with the following steps:

1. Obtain hourly and daily mean tide gauge data from UHSLC
2. Deconstruct the time series into components with different timescales
3. Get high-pass filtered data:
    1. Remove linear trend
    2. Obtain low-frequency SL variability by smoothing daily data with a running Gaussian-shaped low-pass filter (half power period at ~5 months).
    3. Compute seasonal cycle by averaging daily mean WL over all years for each year day
    4. High-frequency SL is the residual of the daily means after removing #3,#4,#5
4. Tidal component is predicted with fits to hourly data
5. Reference all data to MHHW
6. Extreme events are the superposition of seasonal sea level, tidal and high frequency water level components.
    1. Tidal contribution is 95% exceedence threshold of daily highest water above MHHW on each day.
    2. HF variability is the 95% exceedance threshold of the HF filtered data on each year day.
7. Obtain TWL (H95) by stacking the seasonal sea level, 95% tide contribution, and 95% HF contribution

## Deconstruct the time series into components with different timescales.

### Get high-pass filtered data
#### Remove linear trend

In [2]:
# remove the linear trend from rsl_daily

trend_mag, trend_line, trend_rate = process_trend_with_nan(rsl_daily['rsl_mhhw'])

rsl_daily['rsl_mhhw_detrended'] = rsl_daily['rsl_mhhw'] - trend_line

rsl_daily['rsl_mhhw_detrended'].sel(record_id = 590).plot()

NameError: name 'process_trend_with_nan' is not defined

#### Obtain low-frequency SL variability 
We'll do this by smoothing daily data with a running Gaussian-shaped low-pass filter (half power period at ~5 months).

In [ ]:
from scipy.ndimage import gaussian_filter1d
from scipy.signal import detrend

# Define the filter parameters
half_power_period = 5 * 30  # Approximate number of days in 5 months
sigma = half_power_period / np.sqrt(8 * np.log(2))  # Convert half-power period to standard deviation

# Detrend each station's time series and handle NaNs
rsl_mhhw_detrended = np.empty_like(rsl_daily['rsl_mhhw'].values)

for i in range(rsl_daily['record_id'].size):
    data = rsl_daily['rsl_mhhw'][i, :].values
    mask = np.isnan(data)
    data[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data[~mask])
    rsl_mhhw_detrended[i, :] = detrend(data)


# Apply the Gaussian filter to the detrended data
rsl_mhhw_detrended_filtered = gaussian_filter1d(rsl_mhhw_detrended, sigma=sigma, axis=1)

# Add the detrended and filtered data to the dataset
rsl_daily['rsl_mhhw_detrended_interp'] = (('record_id', 'time'), rsl_mhhw_detrended)
rsl_daily['rsl_mhhw_detrended_filtered'] = (('record_id', 'time'), rsl_mhhw_detrended_filtered)



#### Compute the seasonal cycle by averaging daily mean water levels over all years for each year day

In [ ]:
# Compute seasonal cycle by averaging daily rsl_mhhw over all years for each year day
seasonal_cycle = rsl_daily['rsl_mhhw_detrended'].groupby('time.dayofyear').mean(dim='time') 

# Plot the seasonal cycle, with 11 different lines, and add the station names to the legend
plt.figure()
for i in range(seasonal_cycle.shape[0]):
    plt.plot(seasonal_cycle['dayofyear'], seasonal_cycle[i, :], label=rsl_daily['station_name'].values[i], alpha=0.5)
plt.xlabel('Day of Year')
plt.ylabel('Median Detrended rsl_mhhw [m]')
plt.title('Seasonal Cycle of rsl_mhhw Detrended')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

#### Obtain High Frequency data
It is the residual of the daily means after removing the linear trend, low-frequency variability, and seasonal cycle.

In [ ]:
# Extend the seasonal cycle to the full time series length for each station
seasonal_cycle_full = seasonal_cycle.sel(dayofyear=rsl_daily['time'].dt.dayofyear)

# Compute the residual by removing the linear trend, filtered data, and seasonal cycle
residual = rsl_daily['rsl_mhhw_detrended'] - rsl_mhhw_detrended_filtered - seasonal_cycle_full

# Add the residual to the dataset
rsl_daily['rsl_mhhw_high_pass_filtered'] = (('record_id', 'time'), residual.values)

# Plot the residual for each station
plt.figure()
for i in range(rsl_daily['record_id'].size):
    rsl_daily['rsl_mhhw_high_pass_filtered'].sel(record_id=rsl_daily['record_id'][i]).plot(label=rsl_daily['station_name'].values[i])


### Tidal component is predicted with fits to hourly data

In [ ]:
def split_continuous_segments(time, data, max_gap_hours=12):
    # Create a DataFrame from the time and data arrays
    df = pd.DataFrame({'time': time, 'data': data})
    
    # Calculate the time difference in hours
    df['time_diff'] = df['time'].diff().dt.total_seconds() / 3600
    
    # Identify the start of new segments
    df['segment'] = (df['time_diff'] > max_gap_hours).cumsum()
    
    # Group by the segment identifier and convert groups to list of tuples
    segments = [list(zip(group['time'], group['data'])) for _, group in df.groupby('segment')]
    
    return segments

In [ ]:
# convert time in rsl_hourly to fractional year, starting at 0
time = rsl_hourly['time']
time = time - time[0]
time = time/np.timedelta64(1, 'D')
time = time/365.25

In [ ]:
# check to make sure that data_dir/rsl_hawaii_tidal_predictions.nc exists, if not, run the following code to create it
if not (data_dir / 'rsl_hawaii_tidal_predictions.nc').exists():
    print('rsl_hawaii_tidal_predictions.nc not found in ../../data. Will proceed with tidal analysis and prediction.')


    # Prepare an empty array to store tidal predictions for all stations
    sea_level_shape = rsl_hourly['sea_level'].shape
    tidal_predictions = np.full(sea_level_shape, np.nan)  # Initialize with NaNs

    # Perform tidal analysis and prediction for each station
    for i, station_id in enumerate(rsl_hourly['record_id'].values):
        print('Processing ' + str(rsl_hourly['station_name'].sel(record_id=station_id).values))

        sea_level_data = rsl_hourly['sea_level'].sel(record_id=station_id).values
        #remove linear trend
        sea_level_data = sea_level_data - trend_rate.sel(record_id=station_id).values*1000*time
        # print the % nans in the data
        print('Percentage of NaNs in the data: ' + str(np.isnan(sea_level_data).sum() / len(sea_level_data) * 100) + '%')
        time_data = rsl_hourly['time'].values
        latitude = rsl_hourly['lat'].sel(record_id=station_id).values

        # Convert time_data to pandas datetime format for UTide
        time_series = pd.to_datetime(time_data)

        # Split data into continuous segments
        segments = split_continuous_segments(time_series, sea_level_data)
        print('Number of segments: ' + str(len(segments)))

        # Perform harmonic analysis on each segment
        for segment in segments:
            # print counter
            print('Processing segment ' + str(segments.index(segment) + 1) + ' of ' + str(len(segments)))
            segment_time, segment_data = zip(*segment)
            segment_time = np.array(segment_time)
            segment_data = np.array(segment_data)
    
            # Convert datetime to numeric format for interpolation
            segment_time_numeric = segment_time.astype('datetime64[s]').astype(np.float64)
    
            # Check for NaNs and interpolate to fill NaNs if needed
            if np.isnan(segment_data).any():
                mask = np.isnan(segment_data)
                interp_func = interp1d(segment_time_numeric[~mask], segment_data[~mask], kind='linear', fill_value="extrapolate")
                segment_data[mask] = interp_func(segment_time_numeric[mask])
    
            # Perform harmonic analysis
            coef = utide.solve(segment_time, segment_data, lat=latitude)
    
            # Predict the tide using the fitted model
            tide_pred = utide.reconstruct(segment_time, coef)
    
            # Store the tidal predictions in the array
            for t, pred in zip(segment_time, tide_pred.h):
                idx = np.where(time_series == t)[0][0]
                tidal_predictions[i, idx] = pred
    
    # Create the tidal predictions xarray Dataset with the same structure as rsl_hourly
    tidal_predictions_ds = xr.Dataset(
        data_vars={'tidal_prediction': (('record_id', 'time'), tidal_predictions)},
        coords={'time': rsl_hourly['time'].values, 'record_id': rsl_hourly['record_id'].values}
    )

    # save rsl_daily_combined to the data directory
    tidal_predictions_ds.to_netcdf(data_dir / 'rsl_hawaii_tidal_predictions.nc')
    print(f'Tidal predictions saved to: {data_dir / "rsl_hawaii_tidal_predictions.nc"}')

else:
    print('rsl_hawaii_tidal_predictions.nc found in ../../data. Proceed.')

Reference all data to MHHW

In [ ]:
# open rsl_hawaii_tidal_predictions.nc
rsl_hawaii_tidal_predictions = xr.open_dataset(data_dir / 'rsl_hawaii_tidal_predictions.nc')
#

rsl_hawaii_tidal_predictions
#Adjust the tidal precictions to be in MHHW
# for station_id in rsl_hawaii_tidal_predictions.data_vars:
    # rsl_hawaii_tidal_predictions[station_id] = rsl_hawaii_tidal_predictions[station_id] + rsl_hourly['MHHW'].sel(record_id=int(station_id))

In [ ]:
# reference it to MHHW
rsl_hawaii_tidal_predictions['tidal_prediction_mhhw_m'] = 0.001*(rsl_hawaii_tidal_predictions['tidal_prediction'] - rsl_hourly['MHHW'])

#plot the tidal predictions for Honolulu
rid = 4
rsl_hourly['sea_level_mhhw'] = 0.001*(rsl_hourly['sea_level'] - rsl_hourly['MHHW'])
rsl_hourly['sea_level_mhhw'].sel(record_id=rsl_daily.record_id[rid]).plot()
# rsl_daily['rsl_mhhw_detrended_interp'].sel(record_id=rsl_daily.record_id[rid]).plot()
rsl_hawaii_tidal_predictions['tidal_prediction_mhhw_m'].sel(record_id=rsl_daily.record_id[rid]).plot()

#### Find 95% exceedence threshold for tidal component

In [ ]:
# "The tidal contribution is the 95% exceedance threshold of daily highest water above the MHHW on each year-day.""

# first we'll get "daily highest water"
rsl_daily_max_tidal = rsl_hawaii_tidal_predictions['tidal_prediction_mhhw_m']
# .groupby('time.day').max(dim='time')
#plot
# get daily max tidal prediction by resampling to daily and taking the max
rsl_daily_max_tidal = rsl_daily_max_tidal.resample(time='1D').max(dim='time')

#If we go by the exact wording in the Merrifield doc, it's the 95th percentile of the daily max tidal prediction
# and we need to do this on year day
# seasonal_cycle = rsl_daily['rsl_mhhw_detrended'].groupby('time.dayofyear').median(dim='time') #NOTE I AM USING MEDIAN!!!

# # get 95th percentile of rsl_daily_max_tidal
# rsl_daily_max_tidal_95 = rsl_daily_max_tidal.quantile(0.95, dim='time')

# rsl_daily_max_tidal_95

# rsl_daily_max_tidal_95 = rsl_daily_max_tidal.groupby('time.dayofyear').quantile(0.95, dim='time')
# rsl_daily_max_tidal_50 = rsl_daily_max_tidal.groupby('time.dayofyear').quantile(0.5, dim='time')
# rsl_daily_max_tidal_5 = rsl_daily_max_tidal.groupby('time.dayofyear').quantile(0.05, dim='time')


# rsl_daily_max_tidal_95



HF variability is the 95% exceedance threshold of the HF filtered data on each year day.

In [ ]:
# get the 95th percentile of the HF residual data
rsl_daily_residual_95 = rsl_daily['rsl_mhhw_high_pass_filtered'].groupby('time.dayofyear').quantile(0.95, dim='time')

rsl_daily_residual_95

In [ ]:
rsl_low_frequency = rsl_daily['rsl_mhhw_detrended_filtered'].sel(record_id=rsl_daily.record_id[rid]) + trend_line.sel(record_id=rsl_daily.record_id[rid])

# plot rsl_low_frequency, trend_line
rsl_low_frequency.plot()
trend_line.sel(record_id=rsl_daily.record_id[rid]).plot()

Obtain TWL (H95) by stacking the seasonal sea level, tide contribution, and HF contribution

In [ ]:
H95 = seasonal_cycle+ rsl_daily_max_tidal_95+ rsl_daily_residual_95
H95.sel(record_id=570).plot()
# seasonal_cycle.sel(record_id=570).plot()
rsl_daily_max_tidal_95.sel(record_id=570).plot()
rsl_daily_max_tidal_5.sel(record_id=570).plot()

rsl_daily_max_tidal_50.sel(record_id=570).plot()

# rsl_daily_residual_95.sel(record_id=570).plot()
